In [13]:
import pandas as pd
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('vader_lexicon')
import spacy
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer, word_tokenize
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package punkt to /Users/rachana/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/rachana/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/rachana/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
disorder_name = input("Choose one of anxiety, bipolar-disorder ,clinical-depression, post-traumatic-stress-disorder and obsessive-compulsive-disorder")

In [14]:
user_rant=input("Tell us how you feel!")

In [22]:
user_rant

'feeling damn sad think might depressed considered self harm ca sleep'

In [5]:
df=pd.read_excel('mental_health_data.xlsx')
df.head()

,title,tag,Concern,Suggestion
0,Medication experiences,obsessive-compulsive-disorder-ocd,"Hello. I am a 51 y/o man, suffering from Bipo...",drugs.com says their are no interaction issues.
1,Medication experiences,obsessive-compulsive-disorder-ocd,"Hello. I am a 51 y/o man, suffering from Bipo...",I've been on Cymbalta against my will for 9 ye...
2,Depression/OCD crisis,obsessive-compulsive-disorder-ocd,"Trigger warning‚Ä¶,Having a breakdown of crisi...",I feel that. I was for real having a breakdown...
3,Depression/OCD crisis,obsessive-compulsive-disorder-ocd,"Trigger warning‚Ä¶,Having a breakdown of crisi...","I also have OCD so I get it, I really do"
4,Depression/OCD crisis,obsessive-compulsive-disorder-ocd,"Trigger warning‚Ä¶,Having a breakdown of crisi...",üò•üò•‚ù§üíî


In [12]:
def remove_stopwords(text):
    words = nltk.word_tokenize(text)
    stop_words = set(stopwords.words("english"))
    filtered_words = [word for word in words if word.lower() not in stop_words]
    filtered_words = [word for word in filtered_words if word.isalnum()]
    return " ".join(filtered_words)

# Assuming df['Concern'] contains your text data
df['Concern'] = df['Concern'].str.lower()
df['Concern'] = df['Concern'].astype(str).apply(remove_stopwords)
print(df['Concern'])

0        hello 51 man suffering bipolar 2 diabetes arth...
1        hello 51 man suffering bipolar 2 diabetes arth...
2        trigger breakdown crisis stop crying one talk ...
3        trigger breakdown crisis stop crying one talk ...
4        trigger breakdown crisis stop crying one talk ...
                               ...                        
29049    active awhile ice sleeping leaving home anxiet...
29050    active awhile ice sleeping leaving home anxiet...
29051    active awhile ice sleeping leaving home anxiet...
29052    active awhile ice sleeping leaving home anxiet...
29053    active awhile ice sleeping leaving home anxiet...
Name: Concern, Length: 29054, dtype: object


In [15]:
user_rant=remove_stopwords(user_rant)
user_rant

'feeling damn sad think might depressed considered self harm ca sleep'

In [6]:
df_disorder=df[df["tag"]==disorder_name]

In [17]:
df_disorder=df_disorder[:100]

In [18]:
nlp = spacy.load('en_core_web_md')
def word_vector_similarity(text1, text2):
    doc1 = nlp(text1)
    doc2 = nlp(text2)
    return doc1.similarity(doc2)

# Create a new DataFrame with product name, product review, and similarity score
# similarity_score = pd.DataFrame({
#     'Product Name': df['item'],
#     'Concern': df_disorder['Concern'],
#     'Similarity Score': ''
# })

# this adds the simlarity score of each comment in the concern column with the user input
df_disorder['Similarity Score'] = df_disorder['Concern'].apply(word_vector_similarity, text2=user_rant)

/var/folders/nr/1qwqb5150yx0cy6skpt709w80000gn/T/ipykernel_34225/3163476593.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_disorder['Similarity Score'] = df_disorder['Concern'].apply(word_vector_similarity, text2=user_rant)


,title,tag,Concern,Suggestion,Similarity Score
1141,anxiety and,anxiety,beginning to use some not good coping skills. ...,sorry about your stress. stress paralyzes me....,0.689194
1142,anxiety and,anxiety,beginning to use some not good coping skills. ...,"I'm sorry you're struggling. I understand, I d...",0.689194
1143,Depression has come again,anxiety,Hi everyone. I just joined this forum due to w...,You've come to the right place. I'm pretty su...,0.725356
1144,Depression has come again,anxiety,Hi everyone. I just joined this forum due to w...,"Sarahlovesspock, are you still on your medica...",0.725356
1145,Depression has come again,anxiety,Hi everyone. I just joined this forum due to w...,"Hi, I‚Äôve been battling depression my whole l...",0.725356
...,...,...,...,...,...
1236,Anticipatory Anxiety,anxiety,Do you suffer from Anticipatory Anxiety? What ...,Hello I think start with recognising what you ...,0.749805
1237,Anticipatory Anxiety,anxiety,Do you suffer from Anticipatory Anxiety? What ...,"I always have this, so just know that you are ...",0.749805
1238,Anticipatory Anxiety,anxiety,Do you suffer from Anticipatory Anxiety? What ...,Hi LEK8 Anticipatory Anxiety use to control ...,0.749805
1239,Anticipatory Anxiety,anxiety,Do you suffer from Anticipatory Anxiety? What ...,You'll get through that event ok. You/We alw...,0.749805


In [21]:
df_disorder[df_disorder['Similarity Score']>0.5]

,title,tag,Concern,Suggestion,Similarity Score
1141,anxiety and,anxiety,beginning to use some not good coping skills. ...,sorry about your stress. stress paralyzes me....,0.689194
1142,anxiety and,anxiety,beginning to use some not good coping skills. ...,"I'm sorry you're struggling. I understand, I d...",0.689194
1143,Depression has come again,anxiety,Hi everyone. I just joined this forum due to w...,You've come to the right place. I'm pretty su...,0.725356
1144,Depression has come again,anxiety,Hi everyone. I just joined this forum due to w...,"Sarahlovesspock, are you still on your medica...",0.725356
1145,Depression has come again,anxiety,Hi everyone. I just joined this forum due to w...,"Hi, I‚Äôve been battling depression my whole l...",0.725356
...,...,...,...,...,...
1236,Anticipatory Anxiety,anxiety,Do you suffer from Anticipatory Anxiety? What ...,Hello I think start with recognising what you ...,0.749805
1237,Anticipatory Anxiety,anxiety,Do you suffer from Anticipatory Anxiety? What ...,"I always have this, so just know that you are ...",0.749805
1238,Anticipatory Anxiety,anxiety,Do you suffer from Anticipatory Anxiety? What ...,Hi LEK8 Anticipatory Anxiety use to control ...,0.749805
1239,Anticipatory Anxiety,anxiety,Do you suffer from Anticipatory Anxiety? What ...,You'll get through that event ok. You/We alw...,0.749805
